In [1]:
import numpy as np
import pandas as pd
import glob
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
train_folder = r"data/train/*.csv"

test_file = r"data/test/unknown_sample.csv"


In [ ]:
# -------------------
# 1. 슬라이딩 윈도우 함수
# -------------------
def create_windows(data, window_size=500, step=250):
    windows = []
    for start in range(0, len(data)-window_size+1, step):
        end = start + window_size
        windows.append(data[start:end])
    return np.array(windows)

# -------------------
# 2. 훈련 데이터셋 불러오기
# -------------------
X = []
y = []

label_map = {"skin":0, "insulator":1, "metal":2}

for f in glob.glob(train_folder):   # 학습용 CSV는 data/train 폴더
    df = pd.read_csv(f, header=None)
    signal = df.values.flatten()
    
    # 정규화(중요!)
    signal = (signal - np.mean(signal)) / np.std(signal)
    
    # 윈도우로 자르기
    windows = create_windows(signal, window_size=500, step=250)
    
    # 라벨
    for key in label_map:
        if key in f:
            label = label_map[key]
            break
    
    for w in windows:
        X.append(w)
        y.append(label)

X = np.array(X)
y = np.array(y)

# 입력 형태 맞추기
X = np.expand_dims(X, -1)
y = to_categorical(y)

# train/test 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------
# 3. 모델 정의 및 학습
# -------------------
model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], 1)))
model.add(Dense(32, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 모델 저장
model.save("rnn_classifier.h5")

# -------------------
# 4. 새로운 CSV 예측
# -------------------
def classify_csv(filepath, model_path="rnn_classifier.h5"):
    model = load_model(model_path)
    
    df = pd.read_csv(filepath, header=None)
    signal = df.values.flatten()
    
    # 정규화
    signal = (signal - np.mean(signal)) / np.std(signal)
    
    # 윈도우 생성
    windows = create_windows(signal, window_size=500, step=250)
    windows = np.expand_dims(windows, -1)
    
    # 예측
    preds = model.predict(windows)
    pred_classes = np.argmax(preds, axis=1)
    
    # 가장 많이 나온 클래스 → 최종 판정
    final_class = np.bincount(pred_classes).argmax()
    
    inv_label_map = {v:k for k,v in label_map.items()}
    return inv_label_map[final_class]

# -------------------
# 5. 실전 데이터 테스트
# -------------------

result = classify_csv(test_file)
print("이 CSV는:", result)


In [ ]:
# -------------------
# 5. 실전 데이터 테스트
# -------------------
test_file = "data/test/unknown_sample.csv"
result = classify_csv(test_file)
print("이 CSV는:", result)